In [1]:
import cv2 as cv
import numpy as np
import os

Partial occlusion 

In [4]:
def place_cube_random_location(image, cube_size):
    # Get the dimensions of the input image
    height, width, _ = image.shape

    # Generate random coordinates for the top-left corner of the cube
    top_left_x = np.random.randint(0, width - cube_size)
    top_left_y = np.random.randint(0, height - cube_size)

    # Generate random color for the cube (BGR format)
    color = (np.random.randint(0, 256), np.random.randint(0, 256), np.random.randint(0, 256))

    # Draw the cube on the image
    image_with_cube = np.copy(image)
    image_with_cube[top_left_y:top_left_y + cube_size, top_left_x:top_left_x + cube_size, :] = color

    return image_with_cube

Change the illumination (Gamma Transforamtion)

In [5]:
#https://stackoverflow.com/questions/33322488/how-to-change-image-illumination-in-opencv-python

def adjust_gamma(image, gamma=1.0):

   invGamma = 1.0 / gamma
   table = np.array([((i / 255.0) ** invGamma) * 255
      for i in np.arange(0, 256)]).astype("uint8")

   return cv.LUT(image, table)

Add noise to the images 

In [6]:
def noisy(noise_typ,image):
    if noise_typ == "gauss":
        row,col,ch= image.shape
        mean = 0
        var = 1e-2
        #var = 1e-5
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch) * 255
        noisy = image + gauss
        return noisy
        
    elif noise_typ == "s&p":
        row,col,ch = image.shapes
        s_vs_p = 0.5
        amount = 0.00001
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape]
        out[coords] = 255

        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
                for i in image.shape]
        out[coords] = 0
        return out
    elif noise_typ == "poisson":
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        return noisy
    elif noise_typ == "speckle":
        row,col,ch= image.shape
        mean = 0
        var = 1e-2
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)        
        noisy = image + image * gauss
        return noisy

In [7]:
# https://gist.github.com/gutierrezps/f4ddad3bbd2ad5a9b96e3c06378e28b4
def sp_noise(image, prob):
    '''
    Add salt and pepper noise to image
    prob: Probability of the noise
    '''
    output = image.copy()
    if len(image.shape) == 2:
        black = 0
        white = 255            
    else:
        colorspace = image.shape[2]
        if colorspace == 3:  # RGB
            black = np.array([0, 0, 0], dtype='uint8')
            white = np.array([255, 255, 255], dtype='uint8')
        else:  # RGBA
            black = np.array([0, 0, 0, 255], dtype = 'uint8')
            white = np.array([255, 255, 255, 255], dtype = 'uint8')
    probs = np.random.random(output.shape[:2])
    output[probs < (prob / 2)] = black
    output[probs > 1 - (prob / 2)] = white
    return output

In [7]:
def load_images_from_folder_add_noise(folder):
    images = []
    j = 0
    for filename in os.listdir(folder):
        j = j + 1
        img = cv.imread(os.path.join(folder,filename))
        #img = cv.resize(img , (600,600))
        #img = img[20:400,0:400]
        noisyimg = noisy('gauss',img)
        #noisyimg = noisy('speckle',img)
        #noisyimg = sp_noise(img , 1e-2)
        #illimg = adjust_gamma(img, gamma = 2)
        #illimg = adjustThe largest ($0.45$ rad) and smallest range ($0.35$ rad) of error belongs to the second and the last group. The mean (magenta line segment) and median (green line segment) are less than $0.1$ rad for all groups.auss/%s" % filename
        cv.imwrite(imagename , noisyimg)
        if img is not None:
            images.append(img)
    return images

In [8]:
# only run 1 time
folder  = './Dataset/imgs/'
imgs = load_images_from_folder_add_noise(folder) 

Testing the Results for Aditive Noise Images

In [2]:
def remove_files_in_directory(directory):
    # Get the list of files in the directory
    files = os.listdir(directory)

    # Iterate over the files and remove each one
    for file in files:
        file_path = os.path.join(directory, file)
        try:
            if os.path.isfile(file_path):
                os.remove(file_path)
                print(f"Removed: {file_path}")
        except Exception as e:
            print(f"Error removing {file_path}: {e}")

# Specify the directory path
directory_path = "/path/to/your/directory"


In [3]:
remove_files_in_directory('./gauss')

Removed: ./gauss/884.jpg
Removed: ./gauss/387.jpg
Removed: ./gauss/927.jpg
Removed: ./gauss/627.jpg
Removed: ./gauss/384.jpg
Removed: ./gauss/702.jpg
Removed: ./gauss/610.jpg
Removed: ./gauss/45.jpg
Removed: ./gauss/419.jpg
Removed: ./gauss/175.jpg
Removed: ./gauss/926.jpg
Removed: ./gauss/372.jpg
Removed: ./gauss/985.jpg
Removed: ./gauss/770.jpg
Removed: ./gauss/493.jpg
Removed: ./gauss/630.jpg
Removed: ./gauss/578.jpg
Removed: ./gauss/466.jpg
Removed: ./gauss/241.jpg
Removed: ./gauss/943.jpg
Removed: ./gauss/765.jpg
Removed: ./gauss/14.jpg
Removed: ./gauss/298.jpg
Removed: ./gauss/237.jpg
Removed: ./gauss/876.jpg
Removed: ./gauss/700.jpg
Removed: ./gauss/430.jpg
Removed: ./gauss/114.jpg
Removed: ./gauss/153.jpg
Removed: ./gauss/274.jpg
Removed: ./gauss/371.jpg
Removed: ./gauss/720.jpg
Removed: ./gauss/835.jpg
Removed: ./gauss/427.jpg
Removed: ./gauss/1.jpg
Removed: ./gauss/941.jpg
Removed: ./gauss/764.jpg
Removed: ./gauss/178.jpg
Removed: ./gauss/352.jpg
Removed: ./gauss/687.jpg
Remo